# Northwind Data Visualization - Interactive Dashboard

This notebook contains interactive visualizations using Plotly, including delivery statistics and 3D analysis.

In [1]:
# Install required packages if not already installed
%pip install plotly nbformat pandas ipywidgets --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

os.makedirs("../figures", exist_ok=True)

In [3]:
# Load the definitive denormalized dataset
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

data_path = "../data/warehouse/merged_northwind.csv"
if not os.path.exists(data_path):
    print(f"File not found: {data_path}. Please run 'python scripts/main.py' first.")
    df = pd.DataFrame() # Empty fallback
else:
    df = pd.read_csv(data_path)
    df['FullDate'] = pd.to_datetime(df['FullDate'])
    # Clean data for Sunburst (drop rows with missing hierarchy keys)
    df = df.dropna(subset=['Category', 'ProductName'])
    print("Data loaded and cleaned successfully.")
    print(f"Total records available: {len(df)}")
    display(df.head())

Data loaded and cleaned successfully.
Total records available: 58


,OrderId,ProductId,UnitPrice,Quantity,Discount,CustomerId,EmployeeId,DateId,ShippedDate,ShippingFee,...,Country_emp,HomePhone,ProductName,Category,UnitPrice_prod,FullDate,Day,Month,MonthName,Revenue
0,30,34,14.0,100.0,0.0,27,9,20060115,2006-01-22,200.0,...,USA,(123)555-0102,Northwind Traders Beer,Beverages,14.0,2006-01-15,15,1,January,1400.0
1,30,80,3.5,30.0,0.0,27,9,20060115,2006-01-22,200.0,...,USA,(123)555-0102,Northwind Traders Dried Plums,Dried Fruit & Nuts,3.5,2006-01-15,15,1,January,105.0
2,31,7,30.0,10.0,0.0,4,3,20060120,2006-01-22,5.0,...,USA,(123)555-0102,Northwind Traders Dried Pears,Dried Fruit & Nuts,30.0,2006-01-20,20,1,January,300.0
3,31,51,53.0,10.0,0.0,4,3,20060120,2006-01-22,5.0,...,USA,(123)555-0102,Northwind Traders Dried Apples,Dried Fruit & Nuts,53.0,2006-01-20,20,1,January,530.0
4,31,80,3.5,10.0,0.0,4,3,20060120,2006-01-22,5.0,...,USA,(123)555-0102,Northwind Traders Dried Plums,Dried Fruit & Nuts,3.5,2006-01-20,20,1,January,35.0


## 0. Executive KPI Summary


In [4]:
total_revenue = df['Revenue'].sum()
total_orders = df['OrderId'].nunique()
avg_order_value = total_revenue / total_orders if total_orders > 0 else 0

print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Total Orders: {total_orders}")
print(f"Average Order Value: ${avg_order_value:,.2f}")


Total Revenue: $68,137.00
Total Orders: 40
Average Order Value: $1,703.42


## 1. Revenue Breakdown (Sunburst)


In [5]:
fig_sunburst = px.sunburst(
    df, 
    path=['Category', 'ProductName'], 
    values='Revenue', 
    color='Revenue', 
    color_continuous_scale='Viridis',
    title='Product Revenue Composition'
)
fig_sunburst.update_layout(height=700, template='plotly_dark')
fig_sunburst.show()


## 2. Monthly Revenue Trend


In [6]:
df['YearMonth'] = df['FullDate'].dt.to_period('M').astype(str)
monthly_rev = df.groupby('YearMonth')['Revenue'].sum().reset_index()

fig_trend = px.line(
    monthly_rev, 
    x='YearMonth', 
    y='Revenue', 
    title='Monthly Revenue Trajectory',
    markers=True,
    template='plotly_dark'
)
fig_trend.update_traces(line_color='#89b4fa', line_width=3)
fig_trend.show()


## 3. Global Revenue by Country


In [7]:
country_rev = df.groupby('Country')['Revenue'].sum().reset_index().sort_values('Revenue', ascending=False)
fig_country = px.bar(
    country_rev, 
    x='Country', 
    y='Revenue', 
    color='Revenue', 
    title='Top Markets by Revenue',
    template='plotly_dark'
)
fig_country.show()


## 4. Delivery Performance Overview


In [8]:
df['DeliveryStatus'] = df['DeliveredFlag'].map({1: 'Delivered', 0: 'Not Delivered'})
status_counts = df['DeliveryStatus'].value_counts().reset_index()
status_counts.columns = ['Status', 'Count']

fig_pie = px.pie(
    status_counts, 
    values='Count', 
    names='Status', 
    hole=0.4,
    title='Order Fulfillment Status',
    color='Status',
    color_discrete_map={'Delivered': '#2ecc71', 'Not Delivered': '#e74c3c'},
    template='plotly_dark'
)
fig_pie.show()


## 5. Interactive Employee Explorer


In [9]:
import ipywidgets as widgets
from IPython.display import display

df['EmployeeName'] = df['FirstName'].astype(str) + ' ' + df['LastName'].astype(str)
employees = sorted(df['EmployeeName'].unique())

dropdown = widgets.Dropdown(
    options=employees,
    description='Employee:',
    style={'description_width': 'initial'}
)

out = widgets.Output()

def update_chart(change):
    emp_name = change['new']
    with out:
        out.clear_output()
        emp_df = df[df['EmployeeName'] == emp_name]
        fig = px.bar(
            emp_df.groupby('Category')['Revenue'].sum().reset_index(),
            x='Category', y='Revenue', 
            title=f"Revenue Contribution by {emp_name}",
            color='Revenue', template='plotly_dark'
        )
        fig.show()

dropdown.observe(update_chart, names='value')
display(dropdown, out)
if employees: update_chart({'new': employees[0]})


Dropdown(description='Employee:', options=('Andrew Cencini', 'Anne Hellung-Larsen', 'Jan Kotas', 'Laura Giussa…

Output()

## 6. 3D Order Landscape


In [10]:
df['Year'] = df['FullDate'].dt.year
years = sorted(df['Year'].unique())
year_options = ['All'] + [str(y) for y in years]

year_dropdown = widgets.Dropdown(
    options=year_options,
    value='All',
    description='Select Year:',
    style={'description_width': 'initial'}
)

out_3d = widgets.Output()

def update_3d_landscape(change):
    selected_year = change['new']
    with out_3d:
        out_3d.clear_output()
        filtered_df = df if selected_year == 'All' else df[df['Year'] == int(selected_year)]
        
        fig = px.scatter_3d(
            filtered_df,
            x='Country',
            y='FullDate',
            z='Revenue',
            color='DeliveryStatus',
            hover_data=['OrderId', 'ProductName'],
            title=f'Strategic Order Landscape ({selected_year}): Country vs Date vs Revenue',
            template='plotly_dark',
            height=800
        )
        fig.update_layout(margin=dict(l=0, r=0, b=0, t=40))
        fig.show()

year_dropdown.observe(update_3d_landscape, names='value')
display(year_dropdown, out_3d)
update_3d_landscape({'new': 'All'})


Dropdown(description='Select Year:', options=('All', '2006'), style=DescriptionStyle(description_width='initia…

Output()

## 7. Employee-Year Performance (3D Analysis)

This 3D chart analyzes how different employees performed across different years in terms of total revenue.

In [11]:
# Aggregating data for Employee-Year performance
emp_year_rev = df.groupby(['EmployeeName', 'Year'])['Revenue'].sum().reset_index()

fig_emp_3d = px.scatter_3d(
    emp_year_rev,
    x='EmployeeName',
    y='Year',
    z='Revenue',
    color='Revenue',
    size='Revenue',
    title='3D Employee Performance: Name vs Year vs Revenue',
    labels={'Revenue': 'Total Revenue ($)'},
    template='plotly_dark',
    height=800,
    color_continuous_scale='Viridis'
)

fig_emp_3d.update_layout(
    scene=dict(
        xaxis_title='Employee Name',
        yaxis_title='Year',
        zaxis_title='Revenue'
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)
fig_emp_3d.show()


In [12]:
# Alternative 3D View: Orders Count
emp_year_orders = df.groupby(['EmployeeName', 'Year'])['OrderId'].nunique().reset_index()

fig_orders_3d = px.scatter_3d(
    emp_year_orders,
    x='EmployeeName',
    y='Year',
    z='OrderId',
    color='OrderId',
    title='3D Order Volume: Employee vs Year vs Orders',
    template='plotly_dark',
    height=700,
    color_continuous_scale='Plasma'
)
fig_orders_3d.show()
